In [ ]:
# pip install pdfminer.six --trusted-host pypi.python.org --trusted-host pypi.org --trusted-host=files.pythonhosted.org


In [ ]:
# pip install pdfplumber --trusted-host pypi.org --trusted-host pypi.python.org --trusted-host=files.pythonhosted.org

In [1]:
import pandas as pd 
import re
from pdfminer.high_level import extract_text

In [2]:
# Import PDF
text = extract_text('C:\\Users\\fatogunt\\Downloads\\AA731NYC_ASMT.pdf')

# Split text into records
error_records = text.split('\n\n')  

# Process records by splitting based on the pipe ('|') character
parids = []
taxyrs = []  
jurs = []
procedures = []
errmsgs = []

# Process each record
record_lines = []
for record in error_records[1:]:  # Skip the first line (column titles)
    # Check if the line contains '-----', a date, or is empty; if yes, skip
    if '-----' in record or any(char.isdigit() for char in record) or not record.strip():
        print(f"Skipping record: {record}")
        continue
    
    # Concatenate lines until a complete record is obtained
    while '|' not in record:
        record += next(error_records[1:], '')  # Concatenate with the next line

    # Split the record into fields based on the pipe ('|') character
    fields = record.split('|')

    # Check if the number of fields is correct (assuming at least 5 fields)
    if len(fields) >= 5:
        # Extract and process the fields
        parids.append(fields[0].strip())
        taxyrs.append(fields[1].strip())
        jurs.append(fields[2].strip())
        procedures.append(fields[3].strip())
        errmsgs.append(fields[4].strip())
    else:
        # Handle cases where the record does not match the expected structure
        print(f"Skipping record (incorrect structure): {record}")

# Create the DataFrame  
df = pd.DataFrame({
    'PARID': parids, 
    'TAXYR': taxyrs,
    'JUR': jurs,
    'PROCEDURE': procedures,  
    'ERRMSG': errmsgs
})

# Print the first few rows of the DataFrame, excluding records with incorrect structure
print(df[df['PARID'] != 'PARID'].head())

# Save DataFrame to CSV
df.to_csv('error_log.csv', index=False)

Skipping record: PARID     |TAXYR|JUR|PROCEDURE                      |ERRMSG                                                                         
----------|-----|---|-------------------------------|------------------------------------------------------------------------------ 
1000110014| 2021|65 |IASW_EXAPP_VAL.VAL_EXCODE_NYC  |Income must be entered for STAR exemption.. Unit (100011001400000001-0000      
          |     |   |                               |4)                                                                             
Skipping record: 1000110014| 2021|65 |IASW_EXAPP_VAL.VAL_EXCODE_NYC  |Income must be entered for STAR exemption.. Unit (100011001400000001-0000      
          |     |   |                               |7)                                                                             
Skipping record: 1000161437| 2016|65 |IASW_EXAPP_VAL.VAL_EXCODE_NYC  |Income must be entered for STAR exemption.                                     
1000161547| 2015|6

In [3]:
# import pandas as pd

# # Paste the data for the first page here
# page_1_data = """
# 04-DEC-2023 08:59:36 PM ERROR LOG FOR AA401NYC PAGE: 1
# PARID |TAXYR|JUR|PROCEDURE |ERRMSG
# ----------|-----|---|-------------------------------|------------------------------------------------------------------------------
# 1000110014| 2021|65 |IASW_EXAPP_VAL.VAL_EXCODE_NYC |Income must be entered for STAR exemption.. Unit (100011001400000001-00004)
# | | | |4)
# 1000110014| 2021|65 |IASW_EXAPP_VAL.VAL_EXCODE_NYC |Income must be entered for STAR exemption.. Unit (100011001400000001-00007)
# | | | |7)
# """

# # Split the data into lines
# lines = page_1_data.split('\n')

# # Extract header and data
# header = lines[4]
# data_lines = lines[6:-1]  # Exclude header and last empty line

# # Define the columns with leading/trailing spaces removed
# columns = [col.strip() for col in header.split('|')]

# # Create a list to store dictionaries
# data_list = []

# # Process each line and append to the list
# for line in data_lines:
#     # Split the line based on the pipe (|) separator
#     values = [value.strip() for value in line.split('|')]
    
#     # Filter out undesired lines
#     if values[4] and values[4] not in ('4)', '7)'):
#         # Skip lines with empty values
#         if not all(value == '' for value in values):
#             data_dict = {col: val for col, val in zip(columns, values)}
#             data_list.append(data_dict)

# # Create a DataFrame from the list of dictionaries
# df_page_1 = pd.DataFrame(data_list)

# # Display the entire DataFrame without truncation
# pd.set_option("display.max_colwidth", None)
# print(df_page_1)

   1000110014  2021  65  IASW_EXAPP_VAL.VAL_EXCODE_NYC  \
0  1000110014  2021  65  IASW_EXAPP_VAL.VAL_EXCODE_NYC   

   Income must be entered for STAR exemption.. Unit (100011001400000001-00004)  
0  Income must be entered for STAR exemption.. Unit (100011001400000001-00007)  


In [4]:
# # Export DataFrame to a CSV file with an absolute path
# df_page_1.to_csv('C:\\Users\\fatogunt\\Downloads\\output.csv', index=False)

# # Or print individual columns separately
# for index, row in df_page_1.iterrows():
#     print(f"PARID: {row['1000110014']}")
#     print(f"TAXYR: {row['2021']}")
#     print(f"JUR: {row['65']}")
#     print(f"PROCEDURE: {row['IASW_EXAPP_VAL.VAL_EXCODE_NYC']}")
#     print(f"ERRMSG: {row['Income must be entered for STAR exemption.. Unit (100011001400000001-00004)']}\n{'-'*80}")


PARID: 1000110014
TAXYR: 2021
JUR: 65
PROCEDURE: IASW_EXAPP_VAL.VAL_EXCODE_NYC
ERRMSG: Income must be entered for STAR exemption.. Unit (100011001400000001-00007)
--------------------------------------------------------------------------------


In [5]:
import pandas as pd
import pdfplumber

# Load the PDF file
pdf_file_path = 'C:\\Users\\fatogunt\\Downloads\\AA731NYC_ASMT.pdf'
with pdfplumber.open(pdf_file_path) as pdf:
    # Extract text from the first page
    page_1_text = pdf.pages[0].extract_text()

# Split the text into lines
lines = page_1_text.split('\n')

# Extract header and data
header = lines[4]
data_lines = lines[6:-1]  # Exclude header and last empty line

# Create a list to store dictionaries
data_list = []

# Process each line and append to the list
for line in data_lines:
    # Split the line based on the pipe (|) separator
    values = [value.strip() for value in line.split('|')]
    
    # Filter out undesired lines
    if values[4] and values[4] not in ('4)', '7)'):
        data_dict = {col: val for col, val in zip(columns, values)}
        data_list.append(data_dict)

# Create a DataFrame from the list of dictionaries
df_page_1 = pd.DataFrame(data_list)

# Display the DataFrame
print(df_page_1)


    1000110014  2021  65  IASW_EXAPP_VAL.VAL_EXCODE_NYC  \
0   1000161437  2016  65  IASW_EXAPP_VAL.VAL_EXCODE_NYC   
1   1000161547  2015  65  IASW_EXAPP_VAL.VAL_EXCODE_NYC   
2   1000161555  2016  65  IASW_EXAPP_VAL.VAL_EXCODE_NYC   
3   1000161594  2015  65  IASW_EXAPP_VAL.VAL_EXCODE_NYC   
4   1000161595  2015  65  IASW_EXAPP_VAL.VAL_EXCODE_NYC   
5   1000161596  2021  65          CALCXMPNY.CALCPAR_NYC   
6   1000162292  2023  65          Asmt before row Trig.   
7   1000165012  2023  65          CALCXMPNY.CALCPAR_NYC   
8   1000165012  2024  65          CALCXMPNY.CALCPAR_NYC   
9   1000165678  2024  65          CALCXMPNY.CALCPAR_NYC   
10  1000165678  2025  65          CALCXMPNY.CALCPAR_NYC   
11  1000165707  2021  65          CALCXMPNY.CALCPAR_NYC   
12  1000166044  2021  65          CALCXMPNY.CALCPAR_NYC   
13  1000166044  2022  65          CALCXMPNY.CALCPAR_NYC   
14  1000166128  2021  65          CALCXMPNY.CALCPAR_NYC   
15  1000166128  2022  65          CALCXMPNY.CALCPAR_NYC 

In [6]:
import pandas as pd
import pdfplumber

# Load the PDF file
pdf_file_path = 'C:\\Users\\fatogunt\\Downloads\\AA731NYC_ASMT.pdf'
with pdfplumber.open(pdf_file_path) as pdf:
    # Extract text from the first page
    page_1_text = pdf.pages[0].extract_text()

# Split the text into lines
lines = page_1_text.split('\n')

# Find the line that contains the pipe character (|) and use it as the header
header_line = next(line for line in lines if '|' in line)
header = header_line.strip()

# Extract data lines
data_lines = lines[lines.index(header_line) + 1 : -1]  # Exclude header and last empty line

# Create a list to store dictionaries
data_list = []

# Define columns based on the header line
columns = [col.strip() for col in header.split('|')]

# Process each line and append to the list
for line in data_lines:
    # Split the line based on the pipe (|) separator
    values = [value.strip() for value in line.split('|')]
    
    # Filter out undesired lines
    if values[4] and values[4] not in ('4)', '7)'):
        data_dict = {col: val for col, val in zip(columns, values)}
        data_list.append(data_dict)

# Create a DataFrame from the list of dictionaries
df_page_1 = pd.DataFrame(data_list)

# Display the DataFrame
print(df_page_1)


         PARID  TAXYR  JUR                        PROCEDURE  \
0   ----------  -----  ---  -------------------------------   
1   1000110014   2021   65    IASW_EXAPP_VAL.VAL_EXCODE_NYC   
2   1000110014   2021   65    IASW_EXAPP_VAL.VAL_EXCODE_NYC   
3   1000161437   2016   65    IASW_EXAPP_VAL.VAL_EXCODE_NYC   
4   1000161547   2015   65    IASW_EXAPP_VAL.VAL_EXCODE_NYC   
5   1000161555   2016   65    IASW_EXAPP_VAL.VAL_EXCODE_NYC   
6   1000161594   2015   65    IASW_EXAPP_VAL.VAL_EXCODE_NYC   
7   1000161595   2015   65    IASW_EXAPP_VAL.VAL_EXCODE_NYC   
8   1000161596   2021   65            CALCXMPNY.CALCPAR_NYC   
9   1000162292   2023   65            Asmt before row Trig.   
10  1000165012   2023   65            CALCXMPNY.CALCPAR_NYC   
11  1000165012   2024   65            CALCXMPNY.CALCPAR_NYC   
12  1000165678   2024   65            CALCXMPNY.CALCPAR_NYC   
13  1000165678   2025   65            CALCXMPNY.CALCPAR_NYC   
14  1000165707   2021   65            CALCXMPNY.CALCPAR

In [16]:
# import pandas as pd
# import pdfplumber
# import re

# # Function to extract text from PDF using pdfplumber
# def extract_text(pdf_path):
#     with pdfplumber.open(pdf_path) as pdf:
#         text = ''
#         for page in pdf.pages:
#             text += page.extract_text()
#         return text

# # Import PDF
# text = extract_text('C:\\Users\\fatogunt\\Downloads\\AA731NYC_ASMT.pdf')

# # Define a regular expression pattern to capture records
# pattern = re.compile(r'(\d+)\s*\|\s*(\d+)\s*\|\s*(\d+)\s*\|\s*(.+?)\s*\|\s*(.+?)\s*')

# # Find all matches in the text
# matches = pattern.findall(text)

# # Debug prints
# for match in matches:
#     print("Match:", match)

# # Process matches
# parids, taxyrs, jurs, procedures, errmsgs = zip(*matches)

# # Create the DataFrame
# df = pd.DataFrame({
#     'PARID': parids,
#     'TAXYR': taxyrs,
#     'JUR': jurs,
#     'PROCEDURE': procedures,
#     'ERRMSG': errmsgs
# })

# # Print the first few rows of the DataFrame
# print(df.head())

# # Save DataFrame to CSV
# df.to_csv('error_log.csv', index=False)


Match: ('1000110014', '2021', '65', 'IASW_EXAPP_VAL.VAL_EXCODE_NYC', 'I')
Match: ('1000110014', '2021', '65', 'IASW_EXAPP_VAL.VAL_EXCODE_NYC', 'I')
Match: ('1000161437', '2016', '65', 'IASW_EXAPP_VAL.VAL_EXCODE_NYC', 'I')
Match: ('1000161547', '2015', '65', 'IASW_EXAPP_VAL.VAL_EXCODE_NYC', 'I')
Match: ('1000161555', '2016', '65', 'IASW_EXAPP_VAL.VAL_EXCODE_NYC', 'I')
Match: ('1000161594', '2015', '65', 'IASW_EXAPP_VAL.VAL_EXCODE_NYC', 'I')
Match: ('1000161595', '2015', '65', 'IASW_EXAPP_VAL.VAL_EXCODE_NYC', 'I')
Match: ('1000161596', '2021', '65', 'CALCXMPNY.CALCPAR_NYC', 'N')
Match: ('1000162292', '2023', '65', 'Asmt before row Trig.', 'O')
Match: ('1000165012', '2023', '65', 'CALCXMPNY.CALCPAR_NYC', '+')
Match: ('1000165012', '2024', '65', 'CALCXMPNY.CALCPAR_NYC', '+')
Match: ('1000165678', '2024', '65', 'CALCXMPNY.CALCPAR_NYC', '+')
Match: ('1000165678', '2025', '65', 'CALCXMPNY.CALCPAR_NYC', '+')
Match: ('1000165707', '2021', '65', 'CALCXMPNY.CALCPAR_NYC', 'N')
Match: ('1000166044'

In [17]:
# import pandas as pd
# import pdfplumber
# import re

# # Function to extract text from PDF using pdfplumber
# def extract_text(pdf_path):
#     with pdfplumber.open(pdf_path) as pdf:
#         text = ''
#         for page in pdf.pages:
#             text += page.extract_text()
#         return text

# # Import PDF
# text = extract_text('C:\\Users\\fatogunt\\Downloads\\AA731NYC_ASMT.pdf')

# # Define a regular expression pattern to capture records
# pattern = re.compile(r'(\d+)\s*\|\s*(\d+)\s*\|\s*(\d+)\s*\|\s*(.+?)\s*\|\s*(.+?)\s*')

# # Find all matches in the text
# matches = pattern.findall(text)

# # Create the DataFrame directly from matches
# df = pd.DataFrame(matches, columns=['PARID', 'TAXYR', 'JUR', 'PROCEDURE', 'ERRMSG'])

# # Print the first few rows of the DataFrame
# print(df.head())

# # Save DataFrame to CSV
# df.to_csv('error_log.csv', index=False)


        PARID TAXYR JUR                      PROCEDURE ERRMSG
0  1000110014  2021  65  IASW_EXAPP_VAL.VAL_EXCODE_NYC      I
1  1000110014  2021  65  IASW_EXAPP_VAL.VAL_EXCODE_NYC      I
2  1000161437  2016  65  IASW_EXAPP_VAL.VAL_EXCODE_NYC      I
3  1000161547  2015  65  IASW_EXAPP_VAL.VAL_EXCODE_NYC      I
4  1000161555  2016  65  IASW_EXAPP_VAL.VAL_EXCODE_NYC      I
